<a href="https://colab.research.google.com/github/VishalSharma99/Neural-Cipher-Generator/blob/main/Neural_Cipher_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install tensorflow numpy matplotlib

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import random
import string


In [17]:
# Define alphabet
ALPHABET = string.ascii_lowercase


# generate a random substitution cipher key (for training)
def generate_random_cipher():
    key = list(ALPHABET)
    random.shuffle(key)
    return ''.join(key)



# encrypt a message
def encrypt_message(plaintext, key):
    cipher_text = ""
    for char in plaintext:
        if char in ALPHABET:
            index = ALPHABET.index(char)
            cipher_text += key[index]
        else:
            cipher_text += char  # Keep non-alphabetic characters as they are
    return cipher_text

# Decrypt a message using a given cipher key
def decrypt_message(cipher_text, key):
    plain_text = ""
    for char in cipher_text:
        if char in ALPHABET:
            index = key.index(char)
            plain_text += ALPHABET[index]
        else:
            plain_text += char
    return plain_text


In [18]:
# Generate training data for encryption and decryption
def generate_training_data(num_samples=1000):
    data = []
    for _ in range(num_samples):
        plaintext = ''.join(random.choices(ALPHABET, k=20))  # Random plaintext of length 20
        cipher_key = generate_random_cipher()
        cipher_text = encrypt_message(plaintext, cipher_key)
        data.append((plaintext, cipher_text))
    return data

# Prepare the training data
data = generate_training_data()
plaintexts, cipher_texts = zip(*data)

# Convert the characters to numerical values (0-25 for each letter in the alphabet)
def text_to_numeric(text):
    return [ALPHABET.index(c) for c in text]

# Convert the plaintexts and cipher_texts to numeric form
X_train = np.array([text_to_numeric(p) for p in plaintexts])
y_train = np.array([text_to_numeric(c) for c in cipher_texts])


In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed
import random
import string

# Define the alphabet
ALPHABET = string.ascii_lowercase

# Generate a random substitution cipher key (for training)
def generate_random_cipher():
    key = list(ALPHABET)
    random.shuffle(key)
    return ''.join(key)

# Encrypt a message using a given cipher key
def encrypt_message(plaintext, key):
    cipher_text = ""
    for char in plaintext:
        if char in ALPHABET:
            index = ALPHABET.index(char)
            cipher_text += key[index]
        else:
            cipher_text += char  # Keep non-alphabetic characters as they are
    return cipher_text

# Decrypt a message using a given cipher key
def decrypt_message(cipher_text, key):
    plain_text = ""
    for char in cipher_text:
        if char in ALPHABET:
            index = key.index(char)
            plain_text += ALPHABET[index]
        else:
            plain_text += char
    return plain_text

# Generate training data for encryption and decryption
def generate_training_data(num_samples=1000):
    data = []
    for _ in range(num_samples):
        plaintext = ''.join(random.choices(ALPHABET, k=20))  # Random plaintext of length 20
        cipher_key = generate_random_cipher()
        cipher_text = encrypt_message(plaintext, cipher_key)
        data.append((plaintext, cipher_text))
    return data

# Prepare the training data
data = generate_training_data()
plaintexts, cipher_texts = zip(*data)

# Convert the characters to numerical values (0-25 for each letter in the alphabet)
def text_to_numeric(text):
    return [ALPHABET.index(c) for c in text]

# One-hot encode the target labels (ciphertext) for each character
def one_hot_encode(text):
    return np.array([[1 if i == ALPHABET.index(c) else 0 for i in range(26)] for c in text])

# Convert plaintexts and cipher_texts to one-hot encoded form
X_train = np.array([text_to_numeric(p) for p in plaintexts])  # same as before
y_train = np.array([one_hot_encode(c) for c in cipher_texts])

# Ensure the input shape is (num_samples, 20, 1) and target shape is (num_samples, 20, 26)
X_train = X_train.reshape(-1, 20, 1)  # Each character is a separate feature
y_train = y_train.reshape(-1, 20, 26)  # Each character is a 26-dimensional one-hot vector

# Build the neural network model
model = Sequential([
    TimeDistributed(Dense(256, activation='relu'), input_shape=(20, 1)),  # Apply Dense layer to each time step (each character)
    TimeDistributed(Dense(256, activation='relu')),
    TimeDistributed(Dense(26, activation='softmax'))  # Output layer for each character with a 26-length vector
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Generate a new cipher key for a given plaintext
def generate_cipher_from_model(plaintext):
    numeric_plaintext = np.array(text_to_numeric(plaintext)).reshape(1, 20, 1)
    predicted_cipher = model.predict(numeric_plaintext)

    # Convert the predictions back to letters
    cipher_key = [''] * 26
    for i in range(26):
        index = np.argmax(predicted_cipher[0][i])  # Get the predicted index of the letter
        cipher_key[i] = ALPHABET[index]
    return ''.join(cipher_key)

# Test the cipher generation
sample_plaintext = "hello"
cipher_key = generate_cipher_from_model(sample_plaintext)
encrypted_message = encrypt_message(sample_plaintext, cipher_key)
decrypted_message = decrypt_message(encrypted_message, cipher_key)

print(f"Plaintext: {sample_plaintext}")
print(f"Cipher Key: {cipher_key}")
print(f"Encrypted: {encrypted_message}")
print(f"Decrypted: {decrypted_message}")


Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 113ms/step - accuracy: 0.0394 - loss: 3.3511 - val_accuracy: 0.0415 - val_loss: 3.2753
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.0393 - loss: 3.2700 - val_accuracy: 0.0295 - val_loss: 3.2633
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.0417 - loss: 3.2625 - val_accuracy: 0.0430 - val_loss: 3.2602
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0368 - loss: 3.2673 - val_accuracy: 0.0420 - val_loss: 3.2635
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0398 - loss: 3.2661 - val_accuracy: 0.0335 - val_loss: 3.2633
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.0368 - loss: 3.2651 - val_accuracy: 0.0385 - val_loss: 3.2616
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.0389 - loss: 3.2610 - val_accuracy: 0.0340 - val_loss: 3.2607
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.0420 - loss: 3.2610 - val_accuracy: 0.0315 - 

ValueError: cannot reshape array of size 5 into shape (1,20,1)

In [ ]:
# Generate a new cipher key for a given plaintext
def generate_cipher_from_model(plaintext):
    numeric_plaintext = np.array(text_to_numeric(plaintext)).reshape(1, -1)
    predicted_cipher = model.predict(numeric_plaintext)

    # Convert the predictions back to letters
    cipher_key = [''] * 26
    for i in range(26):
        index = np.argmax(predicted_cipher[0][i])  # Get the predicted index of the letter
        cipher_key[i] = ALPHABET[index]
    return ''.join(cipher_key)

# Test the cipher generation
sample_plaintext = "hello"
cipher_key = generate_cipher_from_model(sample_plaintext)
encrypted_message = encrypt_message(sample_plaintext, cipher_key)
decrypted_message = decrypt_message(encrypted_message, cipher_key)

print(f"Plaintext: {sample_plaintext}")
print(f"Cipher Key: {cipher_key}")
print(f"Encrypted: {encrypted_message}")
print(f"Decrypted: {decrypted_message}")


In [ ]:
# Evaluate the model's performance on the validation set
loss, accuracy = model.evaluate(X_train, y_train)
print(f"Model accuracy: {accuracy * 100:.2f}%")


In [ ]:
# Visualize training loss
history = model.history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.show()
